In [2]:
!rm -rf rapidsai-csp-utils
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 535 (delta 174), reused 129 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (535/535), 172.39 KiB | 909.00 KiB/s, done.
Resolving deltas: 100% (276/276), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.1 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 868.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
print(cuml.__version__)

NameError: name 'cuml' is not defined

In [7]:
from cuml.cluster import DBSCAN
from cuml.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_train = x_train.reshape((x_train.shape[0], 28*28)).astype('float32')
X_train = X_train / 255

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_train)

eps_range = np.arange(0.1, 1.0, 0.1)
min_samples_range = range(5, 50, 5)

best_score = -1
best_params = {}

for eps in eps_range:
    for min_sam in min_samples_range:
        model = DBSCAN(eps=eps, min_samples=min_sam)
        pred = model.fit_predict(X_pca)

        if len(set(pred)) > 1:
            score = silhouette_score(X_pca, pred)
            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_sam}

        n_clusters_ = len(set(pred)) - (1 if -1 in pred else 0)
        noise_points = np.sum(pred == -1)
        print(f'eps: {eps}, min_samples: {min_sam}, clusters: {n_clusters_}, noise: {noise_points}')

print("Best Silhouette Score:", best_score)
print("Best Params:", best_params)


[W] [16:25:18.072819] Batch size limited by the chosen integer type (4 bytes). 42221 -> 35791. Using the larger integer type might result in better performance
eps: 0.1, min_samples: 5, clusters: 78, noise: 58277
[W] [16:26:16.494852] Batch size limited by the chosen integer type (4 bytes). 42221 -> 35791. Using the larger integer type might result in better performance
eps: 0.1, min_samples: 10, clusters: 14, noise: 59742
[W] [16:27:15.100273] Batch size limited by the chosen integer type (4 bytes). 42221 -> 35791. Using the larger integer type might result in better performance
eps: 0.1, min_samples: 15, clusters: 1, noise: 59985
[W] [16:28:14.300182] Batch size limited by the chosen integer type (4 bytes). 42221 -> 35791. Using the larger integer type might result in better performance
eps: 0.1, min_samples: 20, clusters: 0, noise: 60000
[W] [16:28:14.681397] Batch size limited by the chosen integer type (4 bytes). 42221 -> 35791. Using the larger integer type might result in better